In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import config

In [21]:
X = torch.load('pos.tensor')
Y = torch.load('labs.tensor')
X.shape

torch.Size([21217, 42])

In [22]:
class_names = config.class_names

In [23]:
num_classes = len(class_names)
model = nn.Sequential(
    nn.Linear(X.shape[1],64),
    nn.ReLU(),
    nn.Linear(64,128),
    nn.ReLU(),
    nn.Linear(128,64),
    nn.ReLU(),
    nn.Linear(64,32),
    nn.ReLU(),
    nn.Linear(32,num_classes),
    nn.Sigmoid()
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [24]:
def OneHot(i):
    global num_classes
    i = i.cpu().numpy()
    l = [[0 for i in range(num_classes)] for j in range(len(i))]
    for x in range(len(i)):
        l[x][i[x]] = 1
    return torch.tensor(l,dtype=torch.float32,device=device)


In [25]:
Y = OneHot(Y)
X = X.to(device)

In [26]:
class MyDataSet(Dataset):
    def __init__(self,X,Y) -> None:
        super().__init__()
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        return X[index],Y[index]
    
    def __len__(self):
        return X.shape[0]

In [27]:
datset = MyDataSet(X,Y)
loader = DataLoader(datset,batch_size=4096,shuffle=True)

In [28]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#model.load_state_dict(torch.load('hand_images_classifier.pth'))

In [29]:
num_epochs = 1000


for epoch in range(num_epochs):
    for inputs,labels in loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch+1)%100 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss:.8f}")
        torch.save(model.state_dict(), 'hand_images_classifier.pth')


Epoch 100/1000, Loss: 0.07422293
Epoch 200/1000, Loss: 0.03206093
Epoch 300/1000, Loss: 0.02309265
Epoch 400/1000, Loss: 0.01529302
Epoch 500/1000, Loss: 0.01508476
Epoch 600/1000, Loss: 0.01207609
Epoch 700/1000, Loss: 0.00974348
Epoch 800/1000, Loss: 0.00859437
Epoch 900/1000, Loss: 0.00892496
Epoch 1000/1000, Loss: 0.00583761


In [30]:
torch.save(model.state_dict(), 'hand_images_classifier.pth')
print("Model saved!")


Model saved!
